In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
L = 4 # length of sequence. lets say -> Hello I am Rishabh.
d_k = 8
d_v = 8

q = torch.randn(L, d_k) #Query -> what am i looking for?
k = torch.randn(L, d_k) #Key -> what i can offer
v = torch.randn(L, d_v) #Value -> what i actually offer

In [87]:
v.dtype

torch.float32

In [ ]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 tensor([[-0.1098,  0.0965, -1.4932,  0.5238,  0.7531, -0.2219,  0.5819, -1.9369],
        [-1.5334, -0.1797, -0.6578, -1.2317, -1.2464, -1.4996, -0.5404,  1.2410],
        [-1.6212, -0.9036,  1.3968,  0.9178,  0.5120, -0.8406, -1.0445,  0.5548],
        [-0.9493,  1.0457, -1.1298, -2.8006,  1.2797,  0.2200,  0.3249,  1.3190]])
K
 tensor([[-0.8497, -0.6987, -0.2052, -0.7812,  0.6873,  0.7836, -1.1109, -3.1063],
        [-0.9898, -0.6022, -0.7153, -0.4674,  0.5514,  2.6549,  1.0582, -0.1468],
        [-0.8913,  0.1938,  1.9682, -0.7404, -0.8666, -0.3064, -0.5359, -0.3575],
        [-1.2399, -1.5235, -0.8159,  1.1373,  0.2193,  0.4134,  0.0062, -0.5673]])
V
 tensor([[-0.1704, -0.3028, -1.2868, -1.3663, -0.0463, -0.6150,  1.2367, -0.8144],
        [ 1.1462, -1.1787, -0.0367,  0.6718,  0.9242,  0.2697,  0.6285, -0.7066],
        [-0.8558,  0.9041, -0.5659,  0.3841, -0.7816, -0.1509,  0.4199,  1.4059],
        [-0.8271,  1.5608, -1.0952,  1.1855,  1.1881,  2.0556,  0.6603, -1.1078]])


In [49]:
torch.matmul(q, k.T)

tensor([[ 5.6369,  1.6001, -3.4142,  2.9788],
        [-2.7607, -2.7506,  2.3348, -0.2896],
        [ 0.1356, -2.4155,  3.5149,  2.7347],
        [-0.9109,  3.8669, -0.9234, -3.0542]])

In [52]:
scaled = torch.matmul(q, k.T)/np.sqrt(d_k)
scaled

tensor([[ 1.9930,  0.5657, -1.2071,  1.0532],
        [-0.9761, -0.9725,  0.8255, -0.1024],
        [ 0.0479, -0.8540,  1.2427,  0.9669],
        [-0.3221,  1.3671, -0.3265, -1.0798]])

In [53]:
q.var(), k.var(), scaled.var()

(tensor(1.2201), tensor(1.1403), tensor(1.0397))

In [56]:
mask = np.tril(np.ones((L, L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [57]:
mask[mask == 0] = -np.inf
mask[mask == 1] = 0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [58]:
masked = scaled + mask
masked

tensor([[ 1.9930,    -inf,    -inf,    -inf],
        [-0.9761, -0.9725,    -inf,    -inf],
        [ 0.0479, -0.8540,  1.2427,    -inf],
        [-0.3221,  1.3671, -0.3265, -1.0798]], dtype=torch.float64)

In [88]:
attention= nn.Softmax(dim = 1)(masked).to(torch.float32)


In [89]:
attention

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4991, 0.5009, 0.0000, 0.0000],
        [0.2124, 0.0862, 0.7014, 0.0000],
        [0.1269, 0.6873, 0.1264, 0.0595]])

In [90]:
new_v = torch.matmul(attention, v)
new_v

tensor([[-0.1704, -0.3028, -1.2868, -1.3663, -0.0463, -0.6150,  1.2367, -0.8144],
        [ 0.4891, -0.7415, -0.6606, -0.3454,  0.4399, -0.1718,  0.9321, -0.7604],
        [-0.5377,  0.4683, -0.6734,  0.0371, -0.4784, -0.2132,  0.6114,  0.7523],
        [ 0.6088, -0.6414, -0.3252,  0.4074,  0.6012,  0.2105,  0.6812, -0.4772]])

In [91]:
v

tensor([[-0.1704, -0.3028, -1.2868, -1.3663, -0.0463, -0.6150,  1.2367, -0.8144],
        [ 1.1462, -1.1787, -0.0367,  0.6718,  0.9242,  0.2697,  0.6285, -0.7066],
        [-0.8558,  0.9041, -0.5659,  0.3841, -0.7816, -0.1509,  0.4199,  1.4059],
        [-0.8271,  1.5608, -1.0952,  1.1855,  1.1881,  2.0556,  0.6603, -1.1078]])

# Multi Head Attention


In [92]:
import torch
from torch import nn
import numpy as np

In [95]:
sequence_length = 4
batch_size = 1
input_dim = 512
d_model = 512
x = torch.randn((batch_size, sequence_length, input_dim))
x.shape

torch.Size([1, 4, 512])

In [96]:
qkv_layer = nn.Linear(in_features = input_dim,
                      out_features = 3 * d_model)

In [99]:
qkv = qkv_layer(x)

In [100]:
qkv.shape

torch.Size([1, 4, 1536])

In [101]:
num_heads = 8
head_dim = d_model // num_heads
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3 * head_dim)
qkv.shape

torch.Size([1, 4, 8, 192])

In [102]:
qkv = qkv.permute(0, 2, 1, 3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [112]:
q, k, v = qkv.chunk(3, dim = -1)
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [124]:
d_k = k.shape[-1]
scaled = torch.matmul(q, k.transpose(3, 2))/np.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [128]:
mask = torch.full(scaled.size(), float('-inf'))
mask = mask.triu(diagonal=1)
mask[0][1]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [129]:
(scaled + mask)[0][0]

tensor([[ 0.4696,    -inf,    -inf,    -inf],
        [ 0.7221,  0.0282,    -inf,    -inf],
        [-0.0234,  0.0427,  0.3621,    -inf],
        [ 0.2339,  0.6128,  0.0339,  0.3698]], grad_fn=<SelectBackward0>)

In [130]:
scaled += mask

In [132]:
attention = nn.Softmax(dim = -1) (scaled)
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.6668, 0.3332, 0.0000, 0.0000],
        [0.2826, 0.3019, 0.4155, 0.0000],
        [0.2260, 0.3301, 0.1850, 0.2589]], grad_fn=<SelectBackward0>)

In [133]:
value = torch.matmul(attention,v)
value.shape

torch.Size([1, 8, 4, 64])

In [135]:
def scaled_dot_product_attention(query, key, value, mask = None):
  d_k = key.shape[-1]
  scaled = torch.matmul(query, key.transpose(-1,2))/np.sqrt(d_k)
  if mask is not None:
    scaled += mask
  attention = nn.Softmax(dim = -1)(scaled)
  values = torch.matmul(attention, value)
  return values, attention

In [137]:
values, attention = scaled_dot_product_attention(q, k, v, mask = mask)

In [138]:
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.6668, 0.3332, 0.0000, 0.0000],
        [0.2826, 0.3019, 0.4155, 0.0000],
        [0.2260, 0.3301, 0.1850, 0.2589]], grad_fn=<SelectBackward0>)

In [139]:
values.shape

torch.Size([1, 8, 4, 64])

In [140]:
values = values.reshape(batch_size, sequence_length, num_heads * head_dim)
values.shape

torch.Size([1, 4, 512])

In [141]:
linear_layer = nn.Linear(in_features = d_model,
                       out_features = d_model)

In [142]:
out = linear_layer(values)
out.shape

torch.Size([1, 4, 512])